In [139]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.base import BaseEstimator
from sklearn.base import RegressorMixin
from sklearn.base import clone
from sklearn.base import TransformerMixin

## Import the data

In [155]:
df=pd.read_csv('final_data_Copy.csv')
print(df.iloc[0].SalePrice_K)
df=df[['m2', 'nb_rooms','nb_bedrooms','nb_bath','apartment_floor','building_floor',
        'Cellar','Parking', 'balc_patio','Renovated', 'Ground_floor',
       'Last_floor', 'district','building_height']]
df.head()

5.794841205603168


,m2,nb_rooms,nb_bedrooms,nb_bath,apartment_floor,building_floor,Cellar,Parking,balc_patio,Renovated,Ground_floor,Last_floor,district,building_height
0,3.496508,1,0,1,1,14,0,1,0,0,0,0,19e,Tower
1,4.804021,5,4,1,2,4,1,1,Terrasse,0,0,0,4e,Average
2,4.219508,4,3,1,0,3,0,0,Terrasse,0,1,0,10e,Small
3,3.583519,2,1,1,2,0,1,0,0,0,0,0,18e,Average
4,3.465736,1,0,1,8,9,1,0,0,0,0,0,15e,Average


## Preprocessing

In [156]:
features=[np.exp(3.49650756146648), 1, 0, 1, 1,14, 'No', 'Yes', 'Nothing', 'No', 'No', 'No', '19e', 20]

In [157]:
def height(x):
    if x ==0:
        return 'Average'
    elif x<4:
        return 'Small'
    elif x<=9:
        return 'Average'
    elif x<=12:
        return 'High'
    else:
        return 'Tower'

In [158]:
def preprocessing(features):
    features[0]=np.log(features[0])
    new_features = [0 if x=='No' else 1 if x=="Yes" else x for x in features]
    new_features[-1]=height(new_features[-1])
    if new_features[8]=='Nothing':
        new_features[8]=0
    df=pd.DataFrame(new_features)
    return new_features
    

In [159]:
my_features=preprocessing(features)
len(my_features)

14

In [161]:
data_in=pd.DataFrame(np.array(my_features).reshape(1,-1),columns=['m2', 'nb_rooms','nb_bedrooms','nb_bath','apartment_floor','building_floor','Cellar','Parking', 'balc_patio','Renovated', 'Ground_floor','Last_floor', 'district','building_height'])

In [163]:
data_in

,m2,nb_rooms,nb_bedrooms,nb_bath,apartment_floor,building_floor,Cellar,Parking,balc_patio,Renovated,Ground_floor,Last_floor,district,building_height
0,3.49650756146648,1,0,1,1,14,0,1,0,0,0,0,19e,Tower


## Load the encoder

In [164]:
import pickle
with open('encoder.p', 'rb') as pickled:
    data = pickle.load(pickled)
    encoder=data['encoder']
        

In [165]:
data_in_enc=pd.DataFrame(encoder.transform(np.array(data_in[['balc_patio','district','building_height']].iloc[0]).reshape(1,-1)))
data_in_enc.columns=encoder.get_feature_names(['balc_patio','district','building_height'])
data_in.drop(['balc_patio','district','building_height'],axis=1,inplace=True)
data_in_enc=pd.DataFrame(np.array(data_in.iloc[0]).reshape(1,-1),columns=['m2', 'nb_rooms','nb_bedrooms','nb_bath','apartment_floor','building_floor',
        'Cellar','Parking','Renovated', 'Ground_floor','Last_floor'])
data_out=pd.concat([df_old,df_enc],axis=1)

In [166]:
data_out

,m2,nb_rooms,nb_bedrooms,nb_bath,apartment_floor,Cellar,Parking,Renovated,Ground_floor,Last_floor,...,district_4e,district_5e,district_6e,district_7e,district_8e,district_9e,building_height_Average,building_height_High,building_height_Small,building_height_Tower
0,3.496508,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [167]:
file_name = "FlaskAPI/models/model_file.p"
with open(file_name, 'rb') as pickled:
       data = pickle.load(pickled)
       model = data['model']
np.exp(model.predict(df_real))

array([328.6])